[`osfclient`](https://github.com/osfclient/osfclient) allows us to store our datasets in OSF and retrieve files using a command-line interface. 

In principle we could use it as a python library: to be looked into.

# Setting up the local directory

`$ echo "export OSF_PASSWORD='your password'" > ~/.osfrc`

`$ source ~/.osf.rc`

Go to the local directory you want to interface with the OSF project

`$ osf init`
> give your username

> give the project id (e.g. 7g42j)

# List OSF project content

`$ osf ls > osf_ls.txt`

`$ head osf_ls.txt`
```osfstorage/randomrot_nodisorder/4v6x_randomrot_copy9_defocus3.0_yes_noise.h5
osfstorage/randomrot_nodisorder/4v6x_randomrot_copy9_defocus2.5_yes_noise.h5
osfstorage/randomrot_nodisorder/4v6x_randomrot_copy99_defocus3.0_yes_noise.h5
osfstorage/randomrot_nodisorder/4v6x_randomrot_copy99_defocus2.5_yes_noise.h5
...```

# Upload to a project

### Upload a single file

Say you want to upload the file `/dir1/subdir1/toto.txt` to the root directory on OSF:

`$ osf upload /dir1/subdir1/toto.txt /`

Now assuming you want to move it to some pre-existing `/dir2/subdir2` directory on OSF:

`$ osf upload /dir1/subdir1/toto.txt /dir2/subdir2/toto.txt`

### Upload a directory

`$ osf upload -r local/directory/ /remote/directory` 

### Advanced usage

To update the content of an existing directory, one first needs to list its current content and upload what is not there yet. We provide here a `bash` script approach to it, assuming proper treatment of the arguments, existence etc. A full working script can be found on our [github](https://github.com/slaclab/cryoEM-notebooks/tree/master/src).

```bash
echo "----------"; echo "OSF CLIENT"; echo "----------"
osfls='osf_ls.txt'
if ! grep -q "$localdir" $osfls; then
  osf upload -r $localdir /
else
  for file in $localdir/*; do
    if ! grep -q "$file" $osfls; then
      osf upload "$file" "/$file"; fi
  done
fi
echo "----------"; echo "OSF CLIENT"; echo "----------"
```


# Download from a project

Careful! The following does not seem to work if the project is Private...

### Download a single file

`$ osf fetch /osfstorage/dir2/subdir2/toto.txt dir1/subdir1/toto.txt`

### Download based on keyword search
There is now recursive option in `osfclient` yet for download purposes. Here is a workaround, again assuming proper treatment of arguments, existence etc. A full working script can be found on our [github](https://github.com/slaclab/cryoEM-notebooks/tree/master/src).

```bash
echo "----------"; echo "OSF CLIENT"; echo "----------"
osfls='osf_ls.txt'
if ! grep -q "$remotedir" $osfls; then
  echo "$remotedir was not found in OSF..."
else
  grep "$remotedir" $osfls > "tmp_${remotedir}.txt"
  while read -r line
  do
    if [ ! -f ${localdir}/${line##*/} ]; then
      echo ">>> $line ..."
      osf fetch $line ${localdir}/${line##*/}
    fi
  done < tmp_${remotedir}.txt
fi
echo "----------"; echo "OSF CLIENT"; echo "----------"
```

### Download whole project

This one is easy: `$ osf clone`